<a href="https://colab.research.google.com/github/Aviral-Aggarwal/Retail_models/blob/master/Retail_xgb_estimations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#New version of numpy is there, using the upgraded one
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

warnings.filterwarnings('ignore')

In [ ]:
input_dir = '/content/gdrive/My Drive/m5-forecasting-accuracy/'

#CHECK AGAIN HOW THIS IS WORKING, IS IT ZIP MECHANISM OR SOMETHING ELSE
#Standard function, copied from stack exchange
def reduce_mem_usage(df, verbose=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
def load_df(file_path):
    t1 = time.time()
    df = pd.read_csv(file_path)
    df = reduce_mem_usage(df=df, verbose=True)
    print('The shape of food_master is ', df.shape)
    t2 = time.time()
    print('\nTime taken to load the DataFrame is:', (t2-t1))
    return df

In [ ]:
#Don't load and run this one, start from loading food_xgb
food = load_df(input_dir + 'food1_master.csv')

Mem. usage decreased to 441.10 Mb (63.6% reduction)
The shape of food_master is  (3530738, 45)

Time taken to load the DataFrame is: 21.360494136810303


In [ ]:
food

,Unnamed: 0,item_id,store_id,id,d,Quantity_sold,wm_yr_wk,sell_price,date,weekday,wday,month,year,mrp,discount,week_number,event_name_1,PresidentsDay,LentStart,LentWeek2,StPatricksDay,Purim End,OrthodoxEaster,Pesach End,Cinco De Mayo,Mother's day,MemorialDay,NBAFinalsStart,NBAFinalsEnd,Father's day,IndependenceDay,Ramadan starts,Eid al-Fitr,LaborDay,ColumbusDay,Halloween,EidAlAdha,VeteransDay,Thanksgiving,Christmas,Chanukah End,NewYear,OrthodoxChristmas,MartinLutherKingDay,Easter
0,0,FOODS_1_001,CA_1,FOODS_1_001_CA_1_validation,d_1,3,11101,2.000000,2011-01-29,Saturday,1,1,2011,2.000000,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,FOODS_1_001,CA_1,FOODS_1_001_CA_1_validation,d_2,0,11101,2.000000,2011-01-30,Sunday,2,1,2011,2.000000,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,FOODS_1_001,CA_1,FOODS_1_001_CA_1_validation,d_3,0,11101,2.000000,2011-01-31,Monday,3,1,2011,2.000000,0.0,5,NaN,-21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,FOODS_1_001,CA_1,FOODS_1_001_CA_1_validation,d_4,1,11101,2.000000,2011-02-01,Tuesday,4,2,2011,2.000000,0.0,5,NaN,-20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,FOODS_1_001,CA_1,FOODS_1_001_CA_1_validation,d_5,4,11101,2.000000,2011-02-02,Wednesday,5,2,2011,2.000000,0.0,5,NaN,-19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3530733,3530733,FOODS_1_219,WI_3,FOODS_1_219_WI_3_validation,d_1909,0,11612,2.240234,2016-04-20,Wednesday,5,4,2016,2.240234,0.0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3530734,3530734,FOODS_1_219,WI_3,FOODS_1_219_WI_3_validation,d_1910,2,11612,2.240234,2016-04-21,Thursday,6,4,2016,2.240234,0.0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3530735,3530735,FOODS_1_219,WI_3,FOODS_1_219_WI_3_validation,d_1911,2,11612,2.240234,2016-04-22,Friday,7,4,2016,2.240234,0.0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3530736,3530736,FOODS_1_219,WI_3,FOODS_1_219_WI_3_validation,d_1912,2,11613,2.240234,2016-04-23,Saturday,1,4,2016,2.240234,0.0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


I am dropping some of the columns for the following reasons:
1. id: combination of item_id and store_id
2. dept_id: name of the category
3. weekday: wday is already present as a number
4. wm_yr_wk_x, wm_yr_wk_y : a representation of the dates
5. d_num: indirectly just sequencing of the time series data
6. event_name_1: already converted into columns
7. date_y: repeat column

In [ ]:
food.drop(food.columns[0], axis=1, inplace=True)

In [ ]:
food.drop(['id', 'd', 'wm_yr_wk', 'weekday', 'date'], axis=1, inplace=True)

In [ ]:
food_try = food.iloc[:, 0:11]

In [ ]:
food_try

,item_id,store_id,Quantity_sold,sell_price,wday,month,year,mrp,discount,week_number,event_name_1
0,FOODS_1_001,CA_1,3,2.000000,1,1,2011,2.000000,0.0,4,NaN
1,FOODS_1_001,CA_1,0,2.000000,2,1,2011,2.000000,0.0,4,NaN
2,FOODS_1_001,CA_1,0,2.000000,3,1,2011,2.000000,0.0,5,NaN
3,FOODS_1_001,CA_1,1,2.000000,4,2,2011,2.000000,0.0,5,NaN
4,FOODS_1_001,CA_1,4,2.000000,5,2,2011,2.000000,0.0,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3530733,FOODS_1_219,WI_3,0,2.240234,5,4,2016,2.240234,0.0,16,NaN
3530734,FOODS_1_219,WI_3,2,2.240234,6,4,2016,2.240234,0.0,16,NaN
3530735,FOODS_1_219,WI_3,2,2.240234,7,4,2016,2.240234,0.0,16,NaN
3530736,FOODS_1_219,WI_3,2,2.240234,1,4,2016,2.240234,0.0,16,NaN


In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [ ]:
y = food_try['Quantity_sold']
X = food_try.drop('Quantity_sold', axis=1)

In [ ]:
X.fillna('none', inplace=True)

In [ ]:
del food

Directly running the below cell on food df also crashes. So I am running it for X

In [ ]:
#X = X.loc[:, ~X.columns.str.contains('^Unnamed')]

In [ ]:
X

,item_id,store_id,sell_price,wday,month,year,mrp,discount,week_number,event_name_1
0,FOODS_1_001,CA_1,2.000000,1,1,2011,2.000000,0.0,4,none
1,FOODS_1_001,CA_1,2.000000,2,1,2011,2.000000,0.0,4,none
2,FOODS_1_001,CA_1,2.000000,3,1,2011,2.000000,0.0,5,none
3,FOODS_1_001,CA_1,2.000000,4,2,2011,2.000000,0.0,5,none
4,FOODS_1_001,CA_1,2.000000,5,2,2011,2.000000,0.0,5,none
...,...,...,...,...,...,...,...,...,...,...
3530733,FOODS_1_219,WI_3,2.240234,5,4,2016,2.240234,0.0,16,none
3530734,FOODS_1_219,WI_3,2.240234,6,4,2016,2.240234,0.0,16,none
3530735,FOODS_1_219,WI_3,2.240234,7,4,2016,2.240234,0.0,16,none
3530736,FOODS_1_219,WI_3,2.240234,1,4,2016,2.240234,0.0,16,none


In [ ]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
d = defaultdict(LabelEncoder)
categorical_mask = (X.dtypes == object)
categorical_columns = X.columns[categorical_mask].tolist()

le = LabelEncoder()
for cat in categorical_columns:
    le.fit(X[cat])
    print(le.classes_)
    X[cat]=le.transform(X[cat])

print(X[categorical_columns].head())

['FOODS_1_001' 'FOODS_1_002' 'FOODS_1_003' 'FOODS_1_004' 'FOODS_1_005'
 'FOODS_1_006' 'FOODS_1_008' 'FOODS_1_009' 'FOODS_1_010' 'FOODS_1_011'
 'FOODS_1_012' 'FOODS_1_013' 'FOODS_1_014' 'FOODS_1_015' 'FOODS_1_016'
 'FOODS_1_017' 'FOODS_1_018' 'FOODS_1_019' 'FOODS_1_020' 'FOODS_1_021'
 'FOODS_1_022' 'FOODS_1_023' 'FOODS_1_024' 'FOODS_1_025' 'FOODS_1_026'
 'FOODS_1_027' 'FOODS_1_028' 'FOODS_1_029' 'FOODS_1_030' 'FOODS_1_031'
 'FOODS_1_032' 'FOODS_1_033' 'FOODS_1_034' 'FOODS_1_035' 'FOODS_1_036'
 'FOODS_1_037' 'FOODS_1_038' 'FOODS_1_039' 'FOODS_1_040' 'FOODS_1_041'
 'FOODS_1_042' 'FOODS_1_043' 'FOODS_1_044' 'FOODS_1_045' 'FOODS_1_046'
 'FOODS_1_047' 'FOODS_1_048' 'FOODS_1_049' 'FOODS_1_050' 'FOODS_1_051'
 'FOODS_1_052' 'FOODS_1_053' 'FOODS_1_054' 'FOODS_1_055' 'FOODS_1_056'
 'FOODS_1_057' 'FOODS_1_058' 'FOODS_1_059' 'FOODS_1_060' 'FOODS_1_061'
 'FOODS_1_062' 'FOODS_1_063' 'FOODS_1_064' 'FOODS_1_065' 'FOODS_1_066'
 'FOODS_1_067' 'FOODS_1_068' 'FOODS_1_069' 'FOODS_1_070' 'FOODS_1_071'
 'FOOD

As per what I read, one hot encoding would be needed for the store_id, and item_id variables, since the numbers assigned to them have no special significance and ordering between them would hurt performance.

For now I am skipping date for two reasons:
1. There are 1919 dates, one hot encoding would be too expensive
2. There are time series data, and for them order might help

In [ ]:
X.shape

(3530738, 10)

Skip the next three cells also and try

In [ ]:
from sklearn.preprocessing import OneHotEncoder
 
ohe = OneHotEncoder(categories='auto', sparse=True)
 
# After applying one hot encoder, the output is no longer a data frame
X_encoded = ohe.fit_transform(X)

In [ ]:
X.shape

(3530738, 10)

In [ ]:
X_encoded.shape

(3530738, 1365)

Try again from here

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
seed = 43
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=seed)

One hot encoding accuracy = 25.88%

only label encoding = 38.45%

In [ ]:
# fit model no training data
model = xgb.XGBRegressor(n_estimators=100, max_depth=10)
model.fit(X_train, y_train)

[14:24:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 38.45%


In [ ]:
print(y_pred)

[0.83422804 2.2923274  0.7298737  ... 0.47951704 0.34490734 0.8183894 ]


In [ ]:
print(y_test)

1037717    1
727555     2
1026395    1
530341     0
380315     2
          ..
642213     0
203028     2
626629     0
471788     1
340520     0
Name: Quantity_sold, Length: 106481, dtype: int16


In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold

In [ ]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 6, 9]
        }

In [ ]:
param_comb = 50
random_search = RandomizedSearchCV(estimator=model, param_distributions=params, n_iter=param_comb, scoring='neg_mean_absolute_error' , n_jobs=-1, random_state=43, verbose=3)

In [ ]:
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
def eval_results(yhat_mean, y):
    temp_dict = {}
    temp_dict['MAD'] = np.nanmean(np.abs(y - yhat_mean))
    temp_dict['MSE'] = np.nanmean(np.square(y - yhat_mean))
    temp_dict['WMAPE'] = np.nansum(np.abs(y - yhat_mean)) / np.nansum(y)
    temp_dict['MD'] = np.nanmean(y - yhat_mean)
    temp_dict['mean'] = np.nanmean(y)
    temp_dict['SMAPE'] = 100. * np.nanmean(np.abs(y - yhat_mean) / ((np.abs(y) + np.abs(yhat_mean)) / 2.))
    # print(temp_dict)
    return temp_dict

In [ ]:
print(eval_results(y_pred, y_test))

{'MAD': 1.2461338, 'MSE': 5.5681224, 'WMAPE': 0.776171239777952, 'MD': -0.002632901, 'mean': 1.6054883030775444, 'SMAPE': 140.56050351326422}


In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=100, max_depth=5)
regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=5, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
y_pred = regressor.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 18.57%


In [ ]:
# imputations and NaN 
# ordered variable 
# Random forest regression
